In [0]:
!git clone https://github.com/Abhinavjha07/ML_Datasets/
!wget http://nlp.stanford.edu/data/glove.6B.zip

from google.colab import drive
drive.mount('/content/drive')

Cloning into 'ML_Datasets'...
remote: Enumerating objects: 20114, done.
remote: Counting objects: 100% (20114/20114), done.
remote: Compressing objects: 100% (20069/20069), done.
remote: Total 60707 (delta 39), reused 20113 (delta 38), pack-reused 40593
Receiving objects: 100% (60707/60707), 1.38 GiB | 46.06 MiB/s, done.
Resolving deltas: 100% (1290/1290), done.
Checking out files: 100% (61107/61107), done.
--2019-05-30 12:47:34--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2019-05-30 12:47:34--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/da

In [0]:
!ls "/content/drive/My Drive/ML_Datasets/genData/MTdata"

com_mt_essays_16thMay.csv  sinc_mt_essays_16thMay.csv
com_mt_mrs_16thMay.csv	   sinc_mt_mrs_16thMay.csv
exc_mt_essays_16thMay.csv  sop_mt_essays_16thMay.csv
exc_mt_mrs_16thMay.csv	   sop_mt_mrs_16thMay.csv


In [0]:

!pip install contractions
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Package cess_esp is already up-to-date!
[nltk_data]    | Downloading packag

True

In [0]:
# Preprocessing the train_data
import os 
import numpy as np
import csv
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.utils import to_categorical,plot_model
from keras.layers import Activation, Dense, Dropout,Input,Add,concatenate
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split,StratifiedKFold
from keras.layers import Conv1D,MaxPooling1D,Embedding,GlobalMaxPooling1D
from keras.initializers import Constant
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
import pickle
!pip install -U nltk
import os
import nltk
nltk.download('all')
from nltk.corpus import stopwords

from tqdm import tqdm
import numpy as np
import os
import sys
from keras.preprocessing.text import Tokenizer,text_to_word_sequence
import re, string, unicodedata
import contractions
import inflect
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer


#preprocessing the texts

def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text

def replace_contractions(text):
    """Replace contractions in string of text"""
    return contractions.fix(text)

def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words

def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

def normalize(words):
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    #words = replace_numbers(words)
    words = remove_stopwords(words)
    words = lemmatize_verbs(words)
    return words


#path of the training files

train_path =  '/content/drive/My Drive/ML_Datasets/genData/MTdata'

#columns which are used for training the model
col_names = ['X.AUTHID','site.content','cSIN_tag','cEXC_tag','cCOM_tag','cRUG_tag','cSOP_tag']

#training file names
file_names = ['com_mt_essays_16thMay.csv','rug_mt_essays_27thMay.csv','exc_mt_essays_16thMay.csv','sinc_mt_essays_16thMay.csv','sop_mt_essays_16thMay.csv']

#file name for ruggedess trait
f_names = ['rug_mt_essays_27thMay.csv']

#processing each file and separating the train text and label
for files in os.listdir(train_path):
    if files in f_names:
        reader = csv.DictReader(open(train_path+'/'+files,encoding='latin-1'))
        
        datalist = []
        for raw in reader:
            datalist.append((raw['X.AUTHID'],raw['site.content'],raw['cSIN_tag'],raw['cEXC_tag'],raw['cCOM_tag'],raw['cRUG_tag'],raw['cSOP_tag']))


train_data = np.array(datalist)
data = pd.DataFrame.from_records(datalist, columns=col_names)
print(train_data.shape)

#pickling the train data
pickle_out = open("train_data_rug.pickle","wb")
pickle.dump(data, pickle_out)
pickle_out.close()


#preprocessing the texts
words = []
for texts in data['site.content']:
    text = denoise_text(texts)
    text = replace_contractions(text)
    word = nltk.word_tokenize(text)
    word = normalize(word)
    words.append(word)

words = np.array(words)
print(words.shape)

new_text = []
for i in range(len(words)):
    text = " ".join(str(x) for x in words[i])
    new_text.append(text)

print(len(new_text))


#pickling the preprocessed text.
pickle_out = open("text_RUG.pickle","wb")
pickle.dump(new_text, pickle_out)
pickle_out.close()


    

Using TensorFlow backend.


     |████████████████████████████████| 1.5MB 4.9MB/s 
  Stored in directory: /root/.cache/pip/wheels/54/40/b7/c56ad418e6cd4d9e1e594b5e138d1ca6eec11a6ee3d464e5bb
Successfully built nltk
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Package cess_esp is already up-to-date!
[nltk_data]    | Downloading packag

(2781, 7)
(2781,)
2781


In [0]:
#Preprocessing the heldout test data

import os 
import numpy as np
import csv
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.utils import to_categorical,plot_model
from keras.layers import Activation, Dense, Dropout,Input,Add,concatenate
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split,StratifiedKFold
from keras.layers import Conv1D,MaxPooling1D,Embedding,GlobalMaxPooling1D
from keras.initializers import Constant
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
import pickle
from nltk.corpus import stopwords
import os
import nltk
nltk.download('all')
nltk.download('wordnet')
from tqdm import tqdm
import numpy as np
import os
import sys
from keras.preprocessing.text import Tokenizer,text_to_word_sequence
import re, string, unicodedata
import contractions
import inflect
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer


def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text

def replace_contractions(text):
    """Replace contractions in string of text"""
    return contractions.fix(text)

def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words

def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

def normalize(words):
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    #words = replace_numbers(words)
    words = remove_stopwords(words)
    words = lemmatize_verbs(words)
    return words

#test data path
test_path = '/content/drive/My Drive/ML_Datasets/genData/heldout_essays_16thMay.csv'
col_names = ['X.AUTHID','TEXT','cSIN','cEXC','cCOM','cRUG','cSOP']

#loading test_data
reader = csv.DictReader(open(test_path,encoding='latin-1'))
        
datalist = []

#creating a array of text files
for raw in reader:
    datalist.append((raw['X.AUTHID'],raw['TEXT'],raw['cSIN'],raw['cEXC'],raw['cCOM'],raw['cRUG'],raw['cSOP']))


train_data = np.array(datalist)
data = pd.DataFrame.from_records(datalist, columns=col_names)
print(train_data.shape)

#pickling the test data
pickle_out = open("test_data.pickle","wb")
pickle.dump(data, pickle_out)
pickle_out.close()


#preprocessing the test texts
words = []
for texts in data['TEXT']:
    text = denoise_text(texts)
    text = replace_contractions(text)
    word = nltk.word_tokenize(text)
    word = normalize(word)
    words.append(word)

words = np.array(words)
print(words.shape)
new_text = []
for i in range(len(words)):
    text = " ".join(str(x) for x in words[i])
    new_text.append(text)
    
print(new_text)

print(len(new_text))


#pickling the test text
pickle_out = open("test_text.pickle","wb")
pickle.dump(new_text, pickle_out)
pickle_out.close()







[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Package cess_esp is already up-to-date!
[nltk_data]    | Downloading packag

In [0]:
#heldout_test_data prediction
import os 
import numpy as np
import csv
import pandas as pd
from keras.preprocessing.text import Tokenizer

from keras.preprocessing.sequence import pad_sequences
from keras.models import Model,load_model
from keras.utils import to_categorical
from keras.layers import Activation, Dense, Dropout,Input,Add,concatenate
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split,StratifiedKFold
from keras.layers import Conv1D,MaxPooling1D,Embedding,GlobalMaxPooling1D
from keras.initializers import Constant
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix,precision_score,recall_score,f1_score
import pickle


#parameters same as the parameters which used for training the model
vocab_size = 30000
batch_size = 128
embedding_dim = 300
max_len = 3000

# /content/drive/My Drive/ML_Datasets/genData/test_data.pickle

#loading the pickled test data
pickle_in = open("test_text.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("test_data.pickle","rb")
data = pickle.load(pickle_in)


#loading the tokenizer pickled during the training of the model
pickle_in = open("tokenizer.pickle","rb")
tokenizer = pickle.load(pickle_in)
# tokenizer = Tokenizer(num_words = vocab_size)
# tokenizer.fit_on_texts(X)
# word_index = tokenizer.word_index

#tokenizing the texts
train_sentences_tokenized = tokenizer.texts_to_sequences(X)

#padding the sequence to have the pre decided length 
X = pad_sequences(train_sentences_tokenized, maxlen=max_len)

tags = ['y','n']

#concatenating the binary laabels to for n_data_samples*10 (2 for each binary label)

label_enc = LabelBinarizer()
label_enc.fit(tags)
Y_1 = label_enc.transform(data['cSIN'])
Y_1 = to_categorical(Y_1)

Y_2 = label_enc.transform(data['cEXC'])
Y_2 = to_categorical(Y_2)

Y_3 = label_enc.transform(data['cCOM'])
Y_3 = to_categorical(Y_3)

Y_4 = label_enc.transform(data['cRUG'])
Y_4 = to_categorical(Y_4)

Y_5 = label_enc.transform(data['cSOP'])
Y_5 = to_categorical(Y_5)

Y = np.concatenate((Y_1,Y_2,Y_3,Y_4,Y_5),axis=1)
print(Y.shape)



# 'cSIN','cEXC','cCOM','cRUG','cSOP'



#loading the trained models
model1 = load_model('my_model_cSIN_tag')
model2 = load_model('my_model_cEXC_tag')
model3 = load_model('my_model_cCOM_tag')
model4 = load_model('my_model_cSOP_tag')
model5 = load_model('my_model_cRUG_tag')


#making test data prediction for all the labels
a,b = 0,2
print('\n\nSincerity')

pred = model1.predict(X)
pred = pred.argmax(axis=1)
c_matrix = confusion_matrix(Y[:,a:b].argmax(axis=1),pred)
print(c_matrix)
accuracy = accuracy_score(Y[:,a:b].argmax(axis=1),pred)
print('Accuracy : ',accuracy)
# precision = true positive / total predicted positive(True positive + False positive)
# recall = true positive / total actual positive(True positive + False Negative)
print(classification_report(Y[:,a:b].argmax(axis=1),pred))


a,b = 2,4

print('\n\nExcitement')

    
pred = model2.predict(X)
pred = pred.argmax(axis=1)
c_matrix = confusion_matrix(Y[:,a:b].argmax(axis=1),pred)
print(c_matrix)
accuracy = accuracy_score(Y[:,a:b].argmax(axis=1),pred)
print('Accuracy : ',accuracy)
# precision = true positive / total predicted positive(True positive + False positive)
# recall = true positive / total actual positive(True positive + False Negative)
print(classification_report(Y[:,a:b].argmax(axis=1),pred))


a,b = 4,6
print('\n\nCompetence')
  
    
pred = model3.predict(X)
pred = pred.argmax(axis=1)
c_matrix = confusion_matrix(Y[:,a:b].argmax(axis=1),pred)
print(c_matrix)
accuracy = accuracy_score(Y[:,a:b].argmax(axis=1),pred)
print('Accuracy : ',accuracy)
# precision = true positive / total predicted positive(True positive + False positive)
# recall = true positive / total actual positive(True positive + False Negative)
print(classification_report(Y[:,a:b].argmax(axis=1),pred))


a,b = 6,8

print('\n\nRuggedness')

    
pred = model5.predict(X)
pred = pred.argmax(axis=1)
c_matrix = confusion_matrix(Y[:,a:b].argmax(axis=1),pred)
print(c_matrix)
accuracy = accuracy_score(Y[:,a:b].argmax(axis=1),pred)
print('Accuracy : ',accuracy)
print('Precision : ',precision_score(Y[:,a:b].argmax(axis=1),pred))
print('Recall : ',recall_score(Y[:,a:b].argmax(axis=1),pred))
print('F1 score : ',f1_score(Y[:,a:b].argmax(axis=1),pred))
# precision = true positive / total predicted positive(True positive + False positive)
# recall = true positive / total actual positive(True positive + False Negative)
print(classification_report(Y[:,a:b].argmax(axis=1),pred))

a,b = 8,10

print('\n\nSophistication')

pred = model4.predict(X)
pred = pred.argmax(axis=1)
c_matrix = confusion_matrix(Y[:,a:b].argmax(axis=1),pred)
print(c_matrix)
accuracy = accuracy_score(Y[:,a:b].argmax(axis=1),pred)
print('Accuracy : ',accuracy)
# precision = true positive / total predicted positive(True positive + False positive)
# recall = true positive / total actual positive(True positive + False Negative)
print(classification_report(Y[:,a:b].argmax(axis=1),pred))

(336, 10)


Ruggedness
[[196   5]
 [123  12]]
Accuracy :  0.6190476190476191
Precision :  0.7058823529411765
Recall :  0.08888888888888889
F1 score :  0.15789473684210528
              precision    recall  f1-score   support

           0       0.61      0.98      0.75       201
           1       0.71      0.09      0.16       135

    accuracy                           0.62       336
   macro avg       0.66      0.53      0.46       336
weighted avg       0.65      0.62      0.51       336



In [0]:
#ht_test_data prediction
import os 
import numpy as np
import csv
import pandas as pd
from keras.preprocessing.text import Tokenizer

from keras.preprocessing.sequence import pad_sequences
from keras.models import Model,load_model
from keras.utils import to_categorical
from keras.layers import Activation, Dense, Dropout,Input,Add,concatenate
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split,StratifiedKFold
from keras.layers import Conv1D,MaxPooling1D,Embedding,GlobalMaxPooling1D
from keras.initializers import Constant
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix,precision_score,recall_score,f1_score
import pickle

#hyperparameters
vocab_size = 30000
batch_size = 128
embedding_dim = 300
max_len = 3000

# content/drive/My Drive/ML_Datasets/genData/test_data.pickle

#loading the pickled test data file
pickle_in = open("ht_test_text.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("ht_test_data.pickle","rb")
data = pickle.load(pickle_in)


#loading the pickled tokenizer
pickle_in = open("tokenizer.pickle","rb")
tokenizer = pickle.load(pickle_in)
# tokenizer = Tokenizer(num_words = vocab_size)
# tokenizer.fit_on_texts(X)
# word_index = tokenizer.word_index

#tokenizing the text data
train_sentences_tokenized = tokenizer.texts_to_sequences(X)

#padding the sequences
X = pad_sequences(train_sentences_tokenized, maxlen=max_len)

tags = ['y','n']


label_enc = LabelBinarizer()
label_enc.fit(tags)

Y_1 = label_enc.transform(data['cSIN'])
Y_1 = to_categorical(Y_1)

Y_2 = label_enc.transform(data['cEXC'])
Y_2 = to_categorical(Y_2)

Y_3 = label_enc.transform(data['cCOM'])
Y_3 = to_categorical(Y_3)

Y_4 = label_enc.transform(data['cRUG'])
Y_4 = to_categorical(Y_4)

Y_5 = label_enc.transform(data['cSOP'])
Y_5 = to_categorical(Y_5)

#concatenating the binary laabels to for n_data_samples*10 (2 for each label [y or n])
Y = np.concatenate((Y_1,Y_2,Y_3,Y_4,Y_5),axis=1)
print(Y.shape)



# 'cSIN','cEXC','cCOM','cRUG','cSOP'

#performing he seven fold validation
kfold = StratifiedKFold(n_splits=7, shuffle=True, random_state=4991)

#list to store the accuracy, precision, recall score and f1-score for the 7-fold validation
acscores1 = []
acscores2 = []
acscores3 = []
acscores4 = []
acscores5 = []

prescores1 = []
prescores2 = []
prescores3 = []
prescores4 = []
prescores5 = []

rescores1 = []
rescores2 = []
rescores3 = []
rescores4 = []
rescores5 = []
fscores1 = []
fscores2 = []
fscores3 = []
fscores4 = []
fscores5 = []

#loading the trained models
model1 = load_model('my_model_cSIN_tag')
model2 = load_model('my_model_cEXC_tag')
model3 = load_model('my_model_cCOM_tag')
model4 = load_model('my_model_cSOP_tag')
model5 = load_model('my_model_cRUG_tag')


#performing the 7-fold validation for all the personality traits
a,b = 0,2
print('\n\nSincerity')
    
for train, test in kfold.split(X, Y[:,a:b].argmax(axis=1)):  
    pred = model1.predict(X[test])
    pred = pred.argmax(axis=1)
    c_matrix = confusion_matrix(Y[test,a:b].argmax(axis=1),pred)
    print(c_matrix)
    accuracy = accuracy_score(Y[test,a:b].argmax(axis=1),pred)
    print('Accuracy : ',accuracy)
    # precision = true positive / total predicted positive(True positive + False positive)
    # recall = true positive / total actual positive(True positive + False Negative)
    print(classification_report(Y[test,a:b].argmax(axis=1),pred))
    acscores1.append(accuracy)
    prescores1.append(precision_score(Y[test,a:b].argmax(axis=1),pred))
    rescores1.append(recall_score(Y[test,a:b].argmax(axis=1),pred))
    fscores1.append(f1_score(Y[test,a:b].argmax(axis=1),pred))

a,b = 2,4

print('\n\nExcitement')
for train, test in kfold.split(X, Y[:,a:b].argmax(axis=1)):   
      pred = model2.predict(X[test])
      pred = pred.argmax(axis=1)
      c_matrix = confusion_matrix(Y[test,a:b].argmax(axis=1),pred)
      print(c_matrix)
      accuracy = accuracy_score(Y[test,a:b].argmax(axis=1),pred)
      print('Accuracy : ',accuracy)
      # # precision = true positive / total predicted positive(True positive + False positive)
      # # recall = true positive / total actual positive(True positive + False Negative)
      print(classification_report(Y[test,a:b].argmax(axis=1),pred))
      acscores2.append(accuracy)
      prescores2.append(precision_score(Y[test,a:b].argmax(axis=1),pred))
      rescores2.append(recall_score(Y[test,a:b].argmax(axis=1),pred))
      fscores2.append(f1_score(Y[test,a:b].argmax(axis=1),pred))

a,b = 4,6
print('\n\nCompetence')
for train, test in kfold.split(X, Y[:,a:b].argmax(axis=1)):   
    
    pred = model3.predict(X[test])
    pred = pred.argmax(axis=1)
    c_matrix = confusion_matrix(Y[test,a:b].argmax(axis=1),pred)
    print(c_matrix)
    accuracy = accuracy_score(Y[test,a:b].argmax(axis=1),pred)
    print('Accuracy : ',accuracy)
    # precision = true positive / total predicted positive(True positive + False positive)
    # recall = true positive / total actual positive(True positive + False Negative)
    print(classification_report(Y[test,a:b].argmax(axis=1),pred))
    acscores3.append(accuracy)
    prescores3.append(precision_score(Y[test,a:b].argmax(axis=1),pred))
    rescores3.append(recall_score(Y[test,a:b].argmax(axis=1),pred))
    fscores3.append(f1_score(Y[test,a:b].argmax(axis=1),pred))
    
a,b = 6,8
print('\n\nRuggedness')
for train, test in kfold.split(X, Y[:,a:b].argmax(axis=1)):   
    
    pred = model5.predict(X[test])
    pred = pred.argmax(axis=1)
    c_matrix = confusion_matrix(Y[test,a:b].argmax(axis=1),pred)
    print(c_matrix)
    accuracy = accuracy_score(Y[test,a:b].argmax(axis=1),pred)
    print('Accuracy : ',accuracy)
    # precision = true positive / total predicted positive(True positive + False positive)
    # recall = true positive / total actual positive(True positive + False Negative)
    print(classification_report(Y[test,a:b].argmax(axis=1),pred))
    acscores5.append(accuracy)
    prescores5.append(precision_score(Y[test,a:b].argmax(axis=1),pred))
    rescores5.append(recall_score(Y[test,a:b].argmax(axis=1),pred))
    fscores5.append(f1_score(Y[test,a:b].argmax(axis=1),pred))

a,b = 8,10

print('\n\nSophistication')
    
for train, test in kfold.split(X, Y[:,a:b].argmax(axis=1)): 
    pred = model4.predict(X[test])
    pred = pred.argmax(axis=1)
    c_matrix = confusion_matrix(Y[test,a:b].argmax(axis=1),pred)
    print(c_matrix)
    accuracy = accuracy_score(Y[test,a:b].argmax(axis=1),pred)
    print('Accuracy : ',accuracy)
    # precision = true positive / total predicted positive(True positive + False positive)
    # recall = true positive / total actual positive(True positive + False Negative)
    print(classification_report(Y[test,a:b].argmax(axis=1),pred))
    acscores4.append(accuracy)
    prescores4.append(precision_score(Y[test,a:b].argmax(axis=1),pred))
    rescores4.append(recall_score(Y[test,a:b].argmax(axis=1),pred))
    fscores4.append(f1_score(Y[test,a:b].argmax(axis=1),pred))

#printing the output for all the personaliy traits
print('\n\nSincerity')   
print(acscores1,'\nMean : ',np.mean(acscores1),'\nStandard deviation : ',np.std(acscores1),'\nPrecision Score : ',np.mean(prescores1),'\nRecall Score : ',np.mean(rescores1),'\nF1 Score : ',np.mean(fscores1))
print('\n\nExcitement')
print(acscores2,'\nMean : ',np.mean(acscores2),'\nStandard deviation : ',np.std(acscores2),'\nPrecision Score : ',np.mean(prescores2),'\nRecall Score : ',np.mean(rescores2),'\nF1 Score : ',np.mean(fscores2))
print('\n\nCompetence')
print(acscores3,'\nMean : ',np.mean(acscores3),'\nStandard deviation : ',np.std(acscores3),'\nPrecision Score : ',np.mean(prescores3),'\nRecall Score : ',np.mean(rescores3),'\nF1 Score : ',np.mean(fscores3))
print('\n\nSophistication')
print(acscores4,'\nMean : ',np.mean(acscores4),'\nStandard deviation : ',np.std(acscores4),'\nPrecision Score : ',np.mean(prescores4),'\nRecall Score : ',np.mean(rescores4),'\nF1 Score : ',np.mean(fscores4))

print('\n\nRuggedness')
print(acscores5,'\nMean : ',np.mean(acscores5),'\nStandard deviation : ',np.std(acscores5),'\nPrecision Score : ',np.mean(prescores5),'\nRecall Score : ',np.mean(rescores5),'\nF1 Score : ',np.mean(fscores5))




FileNotFoundError: ignored

In [0]:
#model training
!pip install imblearn
import os 
import numpy as np
import csv
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.optimizers import Adam
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.utils import to_categorical,plot_model
from keras.layers import Activation, Dense, Dropout,Input,Add,concatenate
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split,StratifiedKFold
from keras.layers import Conv1D,MaxPooling1D,Embedding,GlobalMaxPooling1D
from keras.initializers import Constant
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
import pickle
from imblearn.over_sampling import SMOTE

#loading the training data
pickle_in = open("/content/drive/My Drive/ML_Datasets/genData/text.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("/content/drive/My Drive/ML_Datasets/genData/train_data.pickle","rb")
data = pickle.load(pickle_in)


#forming the embedding matrix
embedding_index = {}
with open('glove.6B.300d.txt') as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs,'f',sep=' ')
        embedding_index[word] = coefs




#hyperparameters
vocab_size = 30000
batch_size = 128
embedding_dim = 300
max_len = 3000


#forming the numerical vector representation of the textual data
tokenizer = Tokenizer(num_words = vocab_size)
tokenizer.fit_on_texts(X)
word_index = tokenizer.word_index
train_sentences_tokenized = tokenizer.texts_to_sequences(X)

X = pad_sequences(train_sentences_tokenized, maxlen=max_len)
# print(X.shape)
# print(X)
# print(tokenizer.word_index)


tags = ['y','n']
label_enc = LabelBinarizer()
label_enc.fit(tags)
Y_1 = label_enc.transform(data['cSIN_tag'])
Y_1 = to_categorical(Y_1)

Y_2 = label_enc.transform(data['cEXC_tag'])
Y_2 = to_categorical(Y_2)
Y_3 = label_enc.transform(data['cCOM_tag'])
Y_3 = to_categorical(Y_3)
Y_4 = label_enc.transform(data['cRUG_tag'])
# Y_rug = Y_4
Y_4 = to_categorical(Y_4)
Y_5 = label_enc.transform(data['cSOP_tag'])

Y_5 = to_categorical(Y_5)

#concatenating the binary laabels to for n_data_samples*10 (2 for each label [y or n])
Y = np.concatenate((Y_1,Y_2,Y_3,Y_4,Y_5),axis=1)


print(Y.shape)
# for rugggedness only
# sm = SMOTE(random_state=4991, n_jobs=8, ratio={1:2500, 0:2500})
# new_X,new_Y = sm.fit_sample(X,Y_rug)
# print(new_X.shape,new_Y.shape)
# new_Y = to_categorical(np.reshape(new_Y,(-1,1)))
###############

#train test split 90% training and 10% validation 
train_X,test_X,train_Y,test_Y = train_test_split(X,Y,test_size = 0.1,random_state = 4991)

print('Preparing embedding matrix')
num_words = min(vocab_size,len(word_index))+1
embedding_matrix = np.zeros((num_words,embedding_dim))

for word,i in word_index.items():
    if i > vocab_size:
        continue

    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector


embedding_layer = Embedding(num_words,
                            embedding_dim,
                            embeddings_initializer = Constant(embedding_matrix),
                            input_length = max_len,
                            trainable = False)

#creating the cnn model

sequence_input = Input(shape = (max_len,),dtype = 'int32')
embedded_sequences = embedding_layer(sequence_input)
x1 = Conv1D(64,1,activation = 'relu')(embedded_sequences)
# x1 = GlobalMaxPooling1D()(x1)
x2 = Conv1D(64,2,activation = 'relu')(embedded_sequences)
# x2 = GlobalMaxPooling1D()(x2)
x3 = Conv1D(64,3,activation = 'relu')(embedded_sequences)
# x3 = GlobalMaxPooling1D()(x3)
print(x1.shape,x2.shape,x3.shape)
x = concatenate([x1,x2,x3],axis=1)
print(x.shape)
x = Dropout(0.5)(x)
x = Conv1D(128,3,activation = 'relu')(x)
docvec = GlobalMaxPooling1D()(x)
doc_model = Model(sequence_input,docvec)
x = Dropout(0.4)(docvec)
x = Dense(64,activation = 'relu')(x)
x = Dropout(0.5)(x)
pred = Dense(2,activation = 'softmax')(x)
model = Model(sequence_input,pred)
# model.summary()

adam = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=0.0001, decay=0.0001)

model.compile(loss = 'binary_crossentropy',optimizer = adam ,metrics = ['accuracy'])
model.summary()
a,b = 0,2
model.fit(train_X,train_Y[:,a:b],batch_size = batch_size,epochs = 100,validation_data = (test_X,test_Y[:,a:b]))
scores = model.evaluate(test_X, test_Y[:,a:b], verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
plot_model(model, to_file='model.png',show_shapes = True)


#saving the model
model.save('my_model_cSIN_tag')
doc_model.save('doc_model_cSIN')

pred = model.predict(test_X)
pred = pred.argmax(axis=1)
    
#printing the metrics
c_matrix = confusion_matrix(test_Y[:,a:b].argmax(axis=1),pred)
print(c_matrix)
accuracy = accuracy_score(test_Y[:,a:b].argmax(axis=1),pred)
print('Accuracy : ',accuracy)
#precision = true positive / total predicted positive(True positive + False positive)
#recall = true positive / total actual positive(True positive + False Negative)
print(classification_report(test_Y[:,a:b].argmax(axis=1),pred))

pickle_out = open("tokenizer.pickle","wb")
pickle.dump(tokenizer, pickle_out)
pickle_out.close()


(5000, 10)
Preparing embedding matrix
(?, 3000, 64) (?, 2999, 64) (?, 2998, 64)
(?, 8997, 64)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 3000)         0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 3000, 300)    9000300     input_4[0][0]                    
__________________________________________________________________________________________________
conv1d_13 (Conv1D)              (None, 3000, 64)     19264       embedding_4[0][0]                
__________________________________________________________________________________________________
conv1d_14 (Conv1D)              (None, 2999, 64)     38464       embedding_4[0][0]                
_______________

In [0]:
#Preprocessing the ht_test data

import os 
import numpy as np
import csv
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.utils import to_categorical,plot_model
from keras.layers import Activation, Dense, Dropout,Input,Add,concatenate
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split,StratifiedKFold
from keras.layers import Conv1D,MaxPooling1D,Embedding,GlobalMaxPooling1D
from keras.initializers import Constant
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
import pickle
from nltk.corpus import stopwords
import os
import nltk
nltk.download('all')
nltk.download('wordnet')
from tqdm import tqdm
import numpy as np
import os
import sys
from keras.preprocessing.text import Tokenizer,text_to_word_sequence
import re, string, unicodedata
import contractions
import inflect
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer


#preprocessing methods
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text

def replace_contractions(text):
    """Replace contractions in string of text"""
    return contractions.fix(text)

def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words

def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

def normalize(words):
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    #words = replace_numbers(words)
    words = remove_stopwords(words)
    words = lemmatize_verbs(words)
    return words
# content/drive/My Drive/ML_Datasets/genData/ht4_essays_data_12thFeb.csv

#path of the hr_test data
test_path = '/content/drive/My Drive/ML_Datasets/genData/ht4_essays_data_12thFeb.csv'
col_names = ['X.AUTHID','TEXT','cSIN','cEXC','cCOM','cRUG','cSOP']

#reading the data
reader = csv.DictReader(open(test_path,encoding='latin-1'))

#creating the test data list
datalist = []
for raw in reader:
    datalist.append((raw['X.AUTHID'],raw['TEXT'],raw['cSIN'],raw['cEXC'],raw['cCOM'],raw['cRUG'],raw['cSOP']))

train_data = np.array(datalist)
data = pd.DataFrame.from_records(datalist, columns=col_names)
print(train_data.shape)

pickle_out = open("ht_test_data.pickle","wb")
pickle.dump(data, pickle_out)
pickle_out.close()


#preprocessing the texts
words = []
for texts in data['TEXT']:
    text = denoise_text(texts)
    text = replace_contractions(text)
    word = nltk.word_tokenize(text)
    word = normalize(word)
    words.append(word)

words = np.array(words)
print(words.shape)
new_text = []
for i in range(len(words)):
    text = " ".join(str(x) for x in words[i])
    new_text.append(text)

print(len(new_text))


#saving the test file
pickle_out = open("ht_test_text.pickle","wb")
pickle.dump(new_text, pickle_out)
pickle_out.close()







[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Package cess_esp is already up-to-date!
[nltk_data]    | Downloading packag

(500, 7)
(500,)
500


In [0]:
# !wget https://dl.fbaipublicfiles.com/fasttext/data/cooking.stackexchange.tar.gz && tar xvzf cooking.stackexchange.tar.gz


!git clone https://github.com/facebookresearch/fastText.git
!cd fastText && mkdir build && cd build && cmake .. && make && make install





Cloning into 'fastText'...
remote: Enumerating objects: 2, done.
remote: Counting objects: 100% (2/2), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3197 (delta 0), reused 1 (delta 0), pack-reused 3195
Receiving objects: 100% (3197/3197), 7.84 MiB | 32.25 MiB/s, done.
Resolving deltas: 100% (2007/2007), done.
-- The C compiler identification is GNU 7.4.0
-- The CXX compiler identification is GNU 7.4.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Configuring done
-- Generating done
-- Build files have been written to

In [0]:
! cd fastText/build && ./fasttext supervised -input /content/cSIN_text.txt -output /content/model_cSIN2 -lr 0.5 -epoch 50 -wordNgrams 3,5 -minCount 2 -ws 20 -loss softmax -verbose 2 -dim 100
! cd fastText/build && ./fasttext supervised -input /content/cEXC_text.txt -output /content/model_cEXC2 -lr 0.5 -epoch 50 -wordNgrams 3,5 -minCount 2 -ws 20 -loss softmax -verbose 2 -dim 100
! cd fastText/build && ./fasttext supervised -input /content/cCOM_text.txt -output /content/model_cCOM2 -lr 0.5 -epoch 50 -wordNgrams 3,5 -minCount 2 -ws 20 -loss softmax -verbose 2 -dim 100
! cd fastText/build && ./fasttext supervised -input /content/cSOP_text.txt -output /content/model_cSOP2 -lr 0.5 -epoch 50 -wordNgrams 3,5 -minCount 2 -ws 20 -loss softmax -verbose 2 -dim 100


Read 13M words
Number of words:  204856
Number of labels: 2
Progress: 100.0% words/sec/thread:   69233 lr:  0.000000 loss:  0.031816 ETA:   0h 0m
Read 13M words
Number of words:  204856
Number of labels: 2
Progress: 100.0% words/sec/thread:   69903 lr:  0.000000 loss:  0.024041 ETA:   0h 0m
Read 13M words
Number of words:  204856
Number of labels: 2
Progress: 100.0% words/sec/thread:   70381 lr:  0.000000 loss:  0.039233 ETA:   0h 0m
Read 13M words
Number of words:  204856
Number of labels: 2
Progress: 100.0% words/sec/thread:   70240 lr:  0.000000 loss:  0.033728 ETA:   0h 0m


In [0]:
! cd fastText/build && ./fasttext test /content/model_cSIN.bin /content/cSIN_heldout_text.txt 1

! cd fastText/build && ./fasttext test /content/model_cEXC.bin /content/cEXC_heldout_text.txt 1

! cd fastText/build && ./fasttext test /content/model_cCOM.bin /content/cCOM_heldout_text.txt 1

! cd fastText/build && ./fasttext test /content/model_cSOP.bin /content/cSOP_heldout_text.txt 1

N	336
P@1	0.286
R@1	0.286
N	336
P@1	0.396
R@1	0.396
N	336
P@1	0.244
R@1	0.244
N	336
P@1	0.622
R@1	0.622


In [0]:
! cd fastText/build && ./fasttext test /content/model_cSIN.bin /content/cSIN_ht_text.txt 1

! cd fastText/build && ./fasttext test /content/model_cEXC.bin /content/cEXC_ht_text.txt 1

! cd fastText/build && ./fasttext test /content/model_cCOM.bin /content/cCOM_ht_text.txt 1

! cd fastText/build && ./fasttext test /content/model_cSOP.bin /content/cSOP_ht_text.txt 1

N	500
P@1	0.374
R@1	0.374
N	500
P@1	0.458
R@1	0.458
N	500
P@1	0.272
R@1	0.272
N	500
P@1	0.674
R@1	0.674


In [0]:
import numpy as np
import pandas as pd
import os 
import csv
train_path =  '/content/drive/My Drive/ML_Datasets/genData/MTdata'
col_names = ['X.AUTHID','site.content','cSIN_tag','cEXC_tag','cCOM_tag','cRUG_tag','cSOP_tag']
file_names = ['com_mt_essays_16thMay.csv','exc_mt_essays_16thMay.csv','sinc_mt_essays_16thMay.csv','sop_mt_essays_16thMay.csv']
x = 0
n_writes = 0
for files in os.listdir(train_path):
    if files in file_names:
        reader = csv.DictReader(open(train_path+'/'+files,encoding='latin-1'))
        
        for raw in reader:
            if x == 0:
                with open('cSIN_text.txt','w') as f:
                    f.write('__label__SINC'+raw['cSIN_tag']+' '+raw['site.content']+'\n')
                    n_writes += 1
                x+=1
            else:
                with open('cSIN_text.txt','a') as f:
                    f.write('__label__SINC'+raw['cSIN_tag']+' '+raw['site.content']+'\n')
                    n_writes += 1
x = 0                   
for files in os.listdir(train_path):
    if files in file_names:
        reader = csv.DictReader(open(train_path+'/'+files,encoding='latin-1'))
        
        for raw in reader:
            if x == 0:
                with open('cEXC_text.txt','w') as f:
                    f.write('__label__EXC'+raw['cEXC_tag']+' '+raw['site.content']+'\n')
                    x+=1
            else:
                with open('cEXC_text.txt','a') as f:
                    f.write('__label__EXC'+raw['cEXC_tag']+' '+raw['site.content']+'\n')
x = 0                    

for files in os.listdir(train_path):
    if files in file_names:
        reader = csv.DictReader(open(train_path+'/'+files,encoding='latin-1'))
        
        for raw in reader:
            if x == 0:
                with open('cCOM_text.txt','w') as f:
                    f.write('__label__COM'+raw['cCOM_tag']+' '+raw['site.content']+'\n')
                    x+=1
            else:
                with open('cCOM_text.txt','a') as f:
                    f.write('__label__COM'+raw['cCOM_tag']+' '+raw['site.content']+'\n')

x=0
for files in os.listdir(train_path):
    if files in file_names:
        reader = csv.DictReader(open(train_path+'/'+files,encoding='latin-1'))
        
        for raw in reader:
            if x == 0:
                with open('cSOP_text.txt','w') as f:
                    f.write('__label__SOP'+raw['cSOP_tag']+' '+raw['site.content']+'\n')
                    x+=1
            else:
                with open('cSOP_text.txt','a') as f:
                    f.write('__label__SOP'+raw['cSOP_tag']+' '+raw['site.content']+'\n')
                    





In [0]:
heldout_test_path = '/content/drive/My Drive/ML_Datasets/genData/heldout_essays_16thMay.csv'

col_names = ['X.AUTHID','TEXT','cSIN','cEXC','cCOM','cRUG','cSOP']

reader = csv.DictReader(open(heldout_test_path,encoding='latin-1'))
x = 0
for raw in reader:
    if x == 0:
        with open('cSIN_heldout_text.txt','w') as f:
            f.write('__label__SINC'+raw['cSIN']+' '+raw['TEXT']+'\n')
        with open('cEXC_heldout_text.txt','w') as f:
            f.write('__label__EXC'+raw['cEXC']+' '+raw['TEXT']+'\n')
        with open('cCOM_heldout_text.txt','w') as f:
            f.write('__label__COM'+raw['cCOM']+' '+raw['TEXT']+'\n')
        with open('cSOP_heldout_text.txt','w') as f:
            f.write('__label__SOP'+raw['cSOP']+' '+raw['TEXT']+'\n')
        x+=1
    else:
        with open('cSIN_heldout_text.txt','a') as f:
            f.write('__label__SINC'+raw['cSIN']+' '+raw['TEXT']+'\n')
        with open('cEXC_heldout_text.txt','a') as f:
            f.write('__label__EXC'+raw['cEXC']+' '+raw['TEXT']+'\n')
        with open('cCOM_heldout_text.txt','a') as f:
            f.write('__label__COM'+raw['cCOM']+' '+raw['TEXT']+'\n')
        with open('cSOP_heldout_text.txt','a') as f:
            f.write('__label__SOP'+raw['cSOP']+' '+raw['TEXT']+'\n')
            
print(x)
print('Done')




1
Done


In [0]:
ht_test_path = '/content/drive/My Drive/ML_Datasets/genData/ht4_essays_data_12thFeb.csv'

col_names = ['X.AUTHID','TEXT','cSIN','cEXC','cCOM','cRUG','cSOP']

reader = csv.DictReader(open(ht_test_path,encoding='latin-1'))
x = 0
for raw in reader:
    if x == 0:
        with open('cSIN_ht_text.txt','w') as f:
            f.write('__label__SINC'+raw['cSIN']+' '+raw['TEXT']+'\n')
        with open('cEXC_ht_text.txt','w') as f:
            f.write('__label__EXC'+raw['cEXC']+' '+raw['TEXT']+'\n')
        with open('cCOM_ht_text.txt','w') as f:
            f.write('__label__COM'+raw['cCOM']+' '+raw['TEXT']+'\n')
        with open('cSOP_ht_text.txt','w') as f:
            f.write('__label__SOP'+raw['cSOP']+' '+raw['TEXT']+'\n')
        x+=1
    else:
        with open('cSIN_ht_text.txt','a') as f:
            f.write('__label__SINC'+raw['cSIN']+' '+raw['TEXT']+'\n')
        with open('cEXC_ht_text.txt','a') as f:
            f.write('__label__EXC'+raw['cEXC']+' '+raw['TEXT']+'\n')
        with open('cCOM_ht_text.txt','a') as f:
            f.write('__label__COM'+raw['cCOM']+' '+raw['TEXT']+'\n')
        with open('cSOP_ht_text.txt','a') as f:
            f.write('__label__SOP'+raw['cSOP']+' '+raw['TEXT']+'\n')
            
print(x)
print('Done')

1
Done


In [0]:

!pip install pyfasttext
from pyfasttext import FastText

model1 = FastText()
model1.supervised(input='cSIN_text.txt', output='model_cSIN', epoch=100, lr=1.0,wordNgrams=2,loss='softmax',dim=100)
print('Done')
model2 = FastText()
model2.supervised(input='cEXC_text.txt', output='model_cEXC', epoch=100, lr=1.0,wordNgrams=2,loss='softmax',dim=100)
print('Done')
model3 = FastText()
model3.supervised(input='cCOM_text.txt', output='model_cCOM', epoch=100, lr=1.0,wordNgrams=2,loss='softmax',dim=100)
print('Done')
model4 = FastText()
model4.supervised(input='cSOP_text.txt', output='model_cSOP', epoch=100, lr=1.0,wordNgrams=2,loss='softmax',dim=100)
print('Done')




     |████████████████████████████████| 245kB 9.1MB/s 
  Stored in directory: /root/.cache/pip/wheels/25/de/c6/3d26a304c069689a7bf5ef2cc774588663700c8381dbf3d947
Successfully built pyfasttext
Done
Done
Done
Done


In [0]:
!pip install pyfasttext
from pyfasttext import FastText
import csv
from sklearn.preprocessing import LabelBinarizer
import pandas as pd
# from sklearn.utils import to_categorical
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix,precision_score,f1_score

sin_model = FastText('model_cSIN2.bin')
exc_model = FastText('model_cEXC2.bin')
com_model = FastText('model_cCOM2.bin')
sop_model = FastText('model_cSOP2.bin')
heldout_test_path = '/content/drive/My Drive/ML_Datasets/genData/heldout_essays_16thMay.csv'
reader = csv.DictReader(open(heldout_test_path,encoding='latin-1'))


output = []
data = []
col_names = ['X.AUTHID','TEXT','cSIN','cEXC','cCOM','cRUG','cSOP']
for raw in reader:
    output.append(sin_model.predict_single(raw['TEXT'],k=1)[0])
    data.append((raw['X.AUTHID'],raw['TEXT'],raw['cSIN'],raw['cEXC'],raw['cCOM'],raw['cRUG'],raw['cSOP']))


data = pd.DataFrame.from_records(data, columns=col_names)

tags = ['SINCn','SINCy']
label_enc = LabelBinarizer()
label_enc.fit(tags)
pred_Y = label_enc.transform(output)


tags = ['n','y']
label_enc = LabelBinarizer()
label_enc.fit(tags)
actual_Y = label_enc.transform(data['cSIN'])

c_matrix = confusion_matrix(actual_Y,pred_Y)
print(c_matrix)
accuracy = accuracy_score(actual_Y,pred_Y)
precision = precision_score(actual_Y,pred_Y)
f = f1_score(actual_Y,pred_Y)
print('Accuracy : ',accuracy)
print('Precision : ',precision)
print('F1_score : ',f)
#precision = true positive / total predicted positive(True positive + False positive)
#recall = true positive / total actual positive(True positive + False Negative)
print(classification_report(actual_Y,pred_Y))




for raw in reader:
    output.append(exc_model.predict_single(raw['TEXT'],k=1)[0])
    
tags = ['EXCn','EXCy']
label_enc = LabelBinarizer()
label_enc.fit(tags)
pred_Y = label_enc.transform(output)

#concatenating the binary labels to for n_data_samples*10 (2 for each label [y or n])
tags = ['n','y']
label_enc = LabelBinarizer()
label_enc.fit(tags)
actual_Y = label_enc.transform(data['cEXC'])

c_matrix = confusion_matrix(actual_Y,pred_Y)
print(c_matrix)
accuracy = accuracy_score(actual_Y,pred_Y)
precision = precision_score(actual_Y,pred_Y)
f = f1_score(actual_Y,pred_Y)
print('Accuracy : ',accuracy)
print('Precision : ',precision)
print('F1_score : ',f)
#precision = true positive / total predicted positive(True positive + False positive)
#recall = true positive / total actual positive(True positive + False Negative)
print(classification_report(actual_Y,pred_Y))

for raw in reader:
    output.append(com_model.predict_single(raw['TEXT'],k=1)[0])
    
tags = ['COMn','COMy']
label_enc = LabelBinarizer()
label_enc.fit(tags)
pred_Y = label_enc.transform(output)


tags = ['n','y']
label_enc = LabelBinarizer()
label_enc.fit(tags)
actual_Y = label_enc.transform(data['cCOM'])

c_matrix = confusion_matrix(actual_Y,pred_Y)
print(c_matrix)
accuracy = accuracy_score(actual_Y,pred_Y)
precision = precision_score(actual_Y,pred_Y)
f = f1_score(actual_Y,pred_Y)
print('Accuracy : ',accuracy)
print('Precision : ',precision)
print('F1_score : ',f)
#precision = true positive / total predicted positive(True positive + False positive)
#recall = true positive / total actual positive(True positive + False Negative)
print(classification_report(actual_Y,pred_Y))

for raw in reader:
    output.append(sop_model.predict_single(raw['TEXT'],k=1)[0])
    
tags = ['SOPn','SOPy']
label_enc = LabelBinarizer()
label_enc.fit(tags)
pred_Y = label_enc.transform(output)


tags = ['n','y']
label_enc = LabelBinarizer()
label_enc.fit(tags)
actual_Y = label_enc.transform(data['cSOP'])

c_matrix = confusion_matrix(actual_Y,pred_Y)
print(c_matrix)
accuracy = accuracy_score(actual_Y,pred_Y)
precision = precision_score(actual_Y,pred_Y)
f = f1_score(actual_Y,pred_Y)
print('Accuracy : ',accuracy)
print('Precision : ',precision)
print('F1_score : ',f)
#precision = true positive / total predicted positive(True positive + False positive)
#recall = true positive / total actual positive(True positive + False Negative)



[[ 66   1]
 [263 170]]
Accuracy :  0.472
Precision :  0.9941520467836257
F1_score :  0.5629139072847683
              precision    recall  f1-score   support

           0       0.20      0.99      0.33        67
           1       0.99      0.39      0.56       433

    accuracy                           0.47       500
   macro avg       0.60      0.69      0.45       500
weighted avg       0.89      0.47      0.53       500

[[161   0]
 [339   0]]
Accuracy :  0.322
Precision :  0.0
F1_score :  0.0
              precision    recall  f1-score   support

           0       0.32      1.00      0.49       161
           1       0.00      0.00      0.00       339

    accuracy                           0.32       500
   macro avg       0.16      0.50      0.24       500
weighted avg       0.10      0.32      0.16       500

[[ 30   0]
 [470   0]]
Accuracy :  0.06
Precision :  0.0
F1_score :  0.0
              precision    recall  f1-score   support

           0       0.06      1.00      0

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: Un

In [0]:
#transfer learning
import os 
import numpy as np
import csv
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.optimizers import Adam
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model,load_model,Sequential
from keras.utils import to_categorical
from keras.layers import Activation, Dense, Dropout,Input,Add,concatenate
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split,StratifiedKFold
from keras.layers import Conv1D,MaxPooling1D,Embedding,GlobalMaxPooling1D
from keras.initializers import Constant
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix,precision_score,recall_score,f1_score
import pickle
from imblearn.over_sampling import SMOTE


vocab_size = 30000
batch_size = 25
embedding_dim = 300
max_len = 3000

#loading the ht_test pickle
pickle_in = open("ht_test_text.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("ht_test_data.pickle","rb")
data = pickle.load(pickle_in)


# pickle_in = open("tokenizer.pickle","rb")
# tokenizer = pickle.load(pickle_in)
tokenizer = Tokenizer(num_words = vocab_size)
tokenizer.fit_on_texts(X)
word_index = tokenizer.word_index
train_sentences_tokenized = tokenizer.texts_to_sequences(X)

X = pad_sequences(train_sentences_tokenized, maxlen=max_len)

tags = ['y','n']

#concatenating the binary labels to for n_data_samples*10 (2 for each label [y or n])
label_enc = LabelBinarizer()
label_enc.fit(tags)
Y_1 = label_enc.transform(data['cSIN'])
Y_1 = to_categorical(Y_1)

Y_2 = label_enc.transform(data['cEXC'])
Y_2 = to_categorical(Y_2)

Y_3 = label_enc.transform(data['cCOM'])
Y_3 = to_categorical(Y_3)

Y_4 = label_enc.transform(data['cRUG'])
Y_4 = to_categorical(Y_4)

Y_5 = label_enc.transform(data['cSOP'])
Y_5 = to_categorical(Y_5)

Y = np.concatenate((Y_1,Y_2,Y_3,Y_4,Y_5),axis=1)
print(Y.shape)



# 'cSIN','cEXC','cCOM','cRUG','cSOP'

#7-fold validation, 6/7 for transfer learning and 1/7 for testing 
kfold = StratifiedKFold(n_splits=7, shuffle=True, random_state=4991)
acscores1 = []
acscores2 = []
acscores3 = []
acscores4 = []
acscores5 = []

prescores1 = []
prescores2 = []
prescores3 = []
prescores4 = []
prescores5 = []

rescores1 = []
rescores2 = []
rescores3 = []
rescores4 = []
rescores5 = []
fscores1 = []
fscores2 = []
fscores3 = []
fscores4 = []
fscores5 = []

#loading the models
model1 = load_model('my_model_cSIN_tag')

model2 = load_model('my_model_cEXC_tag')
model3 = load_model('my_model_cCOM_tag')
model4 = load_model('my_model_cSOP_tag')
model5 = load_model('my_model_cRUG_tag')
a,b = 0,2

#transfer learning on all the personality traits
print('\n\nSincerity')
    
for train, test in kfold.split(X, Y[:,a:b].argmax(axis=1)):
    for layer in model1.layers[:-3]:
        layer.trainable = False
    model = Sequential()
    model.add(model1)
    model.summary()
    adam = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=0.0001, decay=0.0001)
    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy']) 
    model.fit(X[train],Y[train,a:b],epochs = 100,batch_size = batch_size,validation_data = (X[test],Y[test,a:b]))
    model.save('DL_Final_SIN2')
    pred = model.predict(X[test])
    pred = pred.argmax(axis=1)
    c_matrix = confusion_matrix(Y[test,a:b].argmax(axis=1),pred)
    print(c_matrix)
    accuracy = accuracy_score(Y[test,a:b].argmax(axis=1),pred)
    print('Accuracy : ',accuracy)
    # precision = true positive / total predicted positive(True positive + False positive)
    # recall = true positive / total actual positive(True positive + False Negative)
    print(classification_report(Y[test,a:b].argmax(axis=1),pred))
    acscores1.append(accuracy)
    prescores1.append(precision_score(Y[test,a:b].argmax(axis=1),pred))
    rescores1.append(recall_score(Y[test,a:b].argmax(axis=1),pred))
    fscores1.append(f1_score(Y[test,a:b].argmax(axis=1),pred))

a,b = 2,4

print('\n\nExcitement')
for train, test in kfold.split(X, Y[:,a:b].argmax(axis=1)):
      
      for layer in model2.layers[:-3]:
          layer.trainable = False
      model = Sequential()
      model.add(model2)
      model.summary()
      adam = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=0.0001, decay=0.0001)
      model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 
      model.fit(X[train],Y[train,a:b],epochs = 100,batch_size = batch_size,validation_data = (X[test],Y[test,a:b]))
      model.save('DL_Final_EXC2')
      pred = model.predict(X[test])
      pred = pred.argmax(axis=1)
      c_matrix = confusion_matrix(Y[test,a:b].argmax(axis=1),pred)
      print(c_matrix)
      accuracy = accuracy_score(Y[test,a:b].argmax(axis=1),pred)
      print('Accuracy : ',accuracy)
      # # precision = true positive / total predicted positive(True positive + False positive)
      # # recall = true positive / total actual positive(True positive + False Negative)
      print(classification_report(Y[test,a:b].argmax(axis=1),pred))
      acscores2.append(accuracy)
      prescores2.append(precision_score(Y[test,a:b].argmax(axis=1),pred))
      rescores2.append(recall_score(Y[test,a:b].argmax(axis=1),pred))
      fscores2.append(f1_score(Y[test,a:b].argmax(axis=1),pred))

a,b = 4,6
print('\n\nCompetence')
for train, test in kfold.split(X, Y[:,a:b].argmax(axis=1)):   
    
    for layer in model3.layers[:-3]:
        layer.trainable = False
    model = Sequential()
    model.add(model3)
    model.summary()
    adam = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=0.0001, decay=0.0001)
    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy']) 
    model.fit(X[train],Y[train,a:b],epochs = 100,batch_size = batch_size,validation_data = (X[test],Y[test,a:b]))
    model.save('DL_Final_COM2')
    pred = model.predict(X[test])
    pred = pred.argmax(axis=1)
    c_matrix = confusion_matrix(Y[test,a:b].argmax(axis=1),pred)
    print(c_matrix)
    accuracy = accuracy_score(Y[test,a:b].argmax(axis=1),pred)
    print('Accuracy : ',accuracy)
    # precision = true positive / total predicted positive(True positive + False positive)
    # recall = true positive / total actual positive(True positive + False Negative)
    print(classification_report(Y[test,a:b].argmax(axis=1),pred))
    acscores3.append(accuracy)
    prescores3.append(precision_score(Y[test,a:b].argmax(axis=1),pred))
    rescores3.append(recall_score(Y[test,a:b].argmax(axis=1),pred))
    fscores3.append(f1_score(Y[test,a:b].argmax(axis=1),pred))
    
    
a,b = 6,8
print('\n\nRuggedness')


sm = SMOTE(random_state=4991, n_jobs=8, ratio={1:500, 0:500})
for train, test in kfold.split(X, Y[:,a:b].argmax(axis=1)):
    new_Y = Y[train,a:b].argmax(axis=1)
    
    new_X,new_Y = sm.fit_sample(X[train],new_Y)
    new_Y = to_categorical(new_Y)

    for layer in model5.layers[:-3]:
        layer.trainable = False
    model = Sequential()
    model.add(model5)
    model.summary()
    adam = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=0.0001, decay=0.0001)
    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy']) 
    model.fit(new_X,new_Y,epochs = 100,batch_size = batch_size,validation_data = (X[test],Y[test,a:b]))
    model.save('DL_Final_RUG2')
    pred = model.predict(X[test])
    pred = pred.argmax(axis=1)
    c_matrix = confusion_matrix(Y[test,a:b].argmax(axis=1),pred)
    print(c_matrix)
    accuracy = accuracy_score(Y[test,a:b].argmax(axis=1),pred)
    print('Accuracy : ',accuracy)
    # precision = true positive / total predicted positive(True positive + False positive)
    # recall = true positive / total actual positive(True positive + False Negative)
    print(classification_report(Y[test,a:b].argmax(axis=1),pred))
    acscores5.append(accuracy)
    prescores5.append(precision_score(Y[test,a:b].argmax(axis=1),pred))
    rescores5.append(recall_score(Y[test,a:b].argmax(axis=1),pred))
    fscores5.append(f1_score(Y[test,a:b].argmax(axis=1),pred))


a,b = 8,10

print('\n\nSophistication')
    
for train, test in kfold.split(X, Y[:,a:b].argmax(axis=1)):
  
    for layer in model4.layers[:-3]:
        layer.trainable = False
    model = Sequential()
    model.add(model4)
    model.summary()
    adam = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=0.0001, decay=0.0001)
    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy']) 
    model.fit(X[train],Y[train,a:b],epochs = 100,batch_size = batch_size,validation_data = (X[test],Y[test,a:b]))
    model.save('DL_Final_SOP2')
    pred = model.predict(X[test])
    pred = pred.argmax(axis=1)
    c_matrix = confusion_matrix(Y[test,a:b].argmax(axis=1),pred)
    print(c_matrix)
    accuracy = accuracy_score(Y[test,a:b].argmax(axis=1),pred)
    print('Accuracy : ',accuracy)
    # precision = true positive / total predicted positive(True positive + False positive)
    # recall = true positive / total actual positive(True positive + False Negative)
    print(classification_report(Y[test,a:b].argmax(axis=1),pred))
    acscores4.append(accuracy)
    prescores4.append(precision_score(Y[test,a:b].argmax(axis=1),pred))
    rescores4.append(recall_score(Y[test,a:b].argmax(axis=1),pred))
    fscores4.append(f1_score(Y[test,a:b].argmax(axis=1),pred))
    
#printing the results of transfer learning
print('\n\nSincerity')   
print(acscores1,'\nMean : ',np.mean(acscores1),'\nStandard deviation : ',np.std(acscores1),'\nPrecision Score : ',np.mean(prescores1),'\nRecall Score : ',np.mean(rescores1),'\nF1 Score : ',np.mean(fscores1))
print('\n\nExcitement')
print(acscores2,'\nMean : ',np.mean(acscores2),'\nStandard deviation : ',np.std(acscores2),'\nPrecision Score : ',np.mean(prescores2),'\nRecall Score : ',np.mean(rescores2),'\nF1 Score : ',np.mean(fscores2))
print('\n\nCompetence')
print(acscores3,'\nMean : ',np.mean(acscores3),'\nStandard deviation : ',np.std(acscores3),'\nPrecision Score : ',np.mean(prescores3),'\nRecall Score : ',np.mean(rescores3),'\nF1 Score : ',np.mean(fscores3))
print('\n\nSophistication')
print(acscores4,'\nMean : ',np.mean(acscores4),'\nStandard deviation : ',np.std(acscores4),'\nPrecision Score : ',np.mean(prescores4),'\nRecall Score : ',np.mean(rescores4),'\nF1 Score : ',np.mean(fscores4))
print('\n\nRuggedness')
print(acscores5,'\nMean : ',np.mean(acscores5),'\nStandard deviation : ',np.std(acscores5),'\nPrecision Score : ',np.mean(prescores5),'\nRecall Score : ',np.mean(rescores5),'\nF1 Score : ',np.mean(fscores5))






(500, 10)


Sincerity
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_8 (Model)              (None, 2)                 9148782   
Total params: 9,148,782
Trainable params: 8,386
Non-trainable params: 9,140,396
_________________________________________________________________
Train on 428 samples, validate on 72 samples
Epoch 1/100
428/428 [==============================] - 3s 6ms/step - loss: 2.6910 - acc: 0.7103 - val_loss: 1.8244 - val_acc: 0.8611
Epoch 2/100
428/428 [==============================] - 0s 836us/step - loss: 1.1219 - acc: 0.8621 - val_loss: 1.5781 - val_acc: 0.8611
Epoch 3/100
428/428 [==============================] - 0s 820us/step - loss: 0.9825 - acc: 0.8598 - val_loss: 0.9844 - val_acc: 0.8611
Epoch 4/100
428/428 [==============================] - 0s 816us/step - loss: 0.6841 - acc: 0.8528 - val_loss: 1.0864 - val_acc: 0.8611
Epoch 5/100
428/428 [==============================

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Train on 428 samples, validate on 72 samples
Epoch 1/100
428/428 [==============================] - 2s 6ms/step - loss: 0.3946 - acc: 0.8668 - val_loss: 0.4058 - val_acc: 0.8611
Epoch 2/100
428/428 [==============================] - 0s 817us/step - loss: 0.3937 - acc: 0.8668 - val_loss: 0.4009 - val_acc: 0.8611
Epoch 3/100
428/428 [==============================] - 0s 819us/step - loss: 0.4080 - acc: 0.8645 - val_loss: 0.3955 - val_acc: 0.8611
Epoch 4/100
428/428 [==============================] - 0s 822us/step - loss: 0.4194 - acc: 0.8668 - val_loss: 0.3988 - val_acc: 0.8611
Epoch 5/100
428/428 [==============================] - 0s 814us/step - loss: 0.3921 - acc: 0.8668 - val_loss: 0.3999 - val_acc: 0.8611
Epoch 6/100
428/428 [==============================] - 0s 815us/step - loss: 0.4172 - acc: 0.8645 - val_loss: 0.3989 - val_acc: 0.8611
Epoch 7/100
428/428 [==============================] - 0s 817us/step - loss: 0.3913 - acc: 0.8668 - val_loss: 0.4434 - val_acc: 0.8611
Epoch 8/100


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Train on 429 samples, validate on 71 samples
Epoch 1/100
429/429 [==============================] - 3s 7ms/step - loss: 0.6221 - acc: 0.6830 - val_loss: 0.6270 - val_acc: 0.6761
Epoch 2/100
429/429 [==============================] - 0s 821us/step - loss: 0.6054 - acc: 0.6853 - val_loss: 0.6293 - val_acc: 0.6761
Epoch 3/100
429/429 [==============================] - 0s 821us/step - loss: 0.6533 - acc: 0.6713 - val_loss: 0.6318 - val_acc: 0.6761
Epoch 4/100
429/429 [==============================] - 0s 843us/step - loss: 0.6249 - acc: 0.6807 - val_loss: 0.6344 - val_acc: 0.6761
Epoch 5/100
429/429 [==============================] - 0s 824us/step - loss: 0.6224 - acc: 0.6783 - val_loss: 0.6374 - val_acc: 0.6761
Epoch 6/100
429/429 [==============================] - 0s 816us/step - loss: 0.6323 - acc: 0.6783 - val_loss: 0.6520 - val_acc: 0.6761
Epoch 7/100
429/429 [==============================] - 0s 829us/step - loss: 0.6295 - acc: 0.6760 - val_loss: 0.6548 - val_acc: 0.6761
Epoch 8/100


...

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_2 (Model)              (None, 2)                 9148782   
Total params: 9,148,782
Trainable params: 8,386
Non-trainable params: 9,140,396
_________________________________________________________________
Train on 1000 samples, validate on 71 samples
Epoch 1/100
1000/1000 [==============================] - 5s 5ms/step - loss: 0.6961 - acc: 0.4830 - val_loss: 0.6930 - val_acc: 0.6197
Epoch 2/100
1000/1000 [==============================] - 1s 761us/step - loss: 0.6934 - acc: 0.4970 - val_loss: 0.6926 - val_acc: 0.6197
Epoch 3/100
1000/1000 [==============================] - 1s 762us/step - loss: 0.6934 - acc: 0.4880 - val_loss: 0.6939 - val_acc: 0.3803
Epoch 4/100
1000/1000 [==============================] - 1s 763us/step - loss: 0.6938 - acc: 0.5000 - val_loss: 0.6947 - val_acc: 0.3803
Epoch 5/100
1000/1000 [==============================] - 1s 764u

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/imblearn/utils/_validation.py:257: UserWarning: After over-sampling, the number of samples (500) in class 1 will be larger than the number of samples in the majority class (class #0 -> 266)
  n_samples_majority))
/usr/local/lib/python3.6/dist-packages/imblearn/utils/_validation.py:257: UserWarning: After over-sampling, the number of samples (500) in class 0 will be larger than the number of samples in the majority class (class #0 -> 266)
  n_samples_majority))


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_2 (Model)              (None, 2)                 9148782   
Total params: 9,148,782
Trainable params: 8,386
Non-trainable params: 9,140,396
_________________________________________________________________
Train on 1000 samples, validate on 71 samples
Epoch 1/100
1000/1000 [==============================] - 5s 5ms/step - loss: 0.6941 - acc: 0.4980 - val_loss: 0.6888 - val_acc: 0.6197
Epoch 2/100
1000/1000 [==============================] - 1s 759us/step - loss: 0.6941 - acc: 0.4620 - val_loss: 0.6897 - val_acc: 0.6197
Epoch 3/100
1000/1000 [==============================] - 1s 769us/step - loss: 0.6936 - acc: 0.5010 - val_loss: 0.6938 - val_acc: 0.3803
Epoch 4/100
1000/1000 [==============================] - 1s 769us/step - loss: 0.6935 - acc: 0.4860 - val_loss: 0.6922 - val_acc: 0.6197
Epoch 5/100
1000/1000 [==============================] - 1s 769u

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Train on 428 samples, validate on 72 samples
Epoch 1/100
428/428 [==============================] - 5s 11ms/step - loss: 1.2146 - acc: 0.5444 - val_loss: 0.6647 - val_acc: 0.5556
Epoch 2/100
428/428 [==============================] - 0s 825us/step - loss: 0.6797 - acc: 0.5537 - val_loss: 0.6712 - val_acc: 0.5556
Epoch 3/100
428/428 [==============================] - 0s 836us/step - loss: 0.6942 - acc: 0.5514 - val_loss: 0.6728 - val_acc: 0.5556
Epoch 4/100
428/428 [==============================] - 0s 830us/step - loss: 0.6895 - acc: 0.5140 - val_loss: 0.6755 - val_acc: 0.5556
Epoch 5/100
428/428 [==============================] - 0s 824us/step - loss: 0.6825 - acc: 0.5514 - val_loss: 0.6698 - val_acc: 0.5556
Epoch 6/100
428/428 [==============================] - 0s 822us/step - loss: 0.6798 - acc: 0.5514 - val_loss: 0.6700 - val_acc: 0.5556
Epoch 7/100
428/428 [==============================] - 0s 823us/step - loss: 0.6770 - acc: 0.5514 - val_loss: 0.6725 - val_acc: 0.5556
Epoch 8/100

In [0]:
#transfer learning on heldout

import os 
import numpy as np
import csv
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.optimizers import Adam
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model,load_model,Sequential
from keras.utils import to_categorical
from keras.layers import Activation, Dense, Dropout,Input,Add,concatenate
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split,StratifiedKFold
from keras.layers import Conv1D,MaxPooling1D,Embedding,GlobalMaxPooling1D
from keras.initializers import Constant
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix,precision_score,recall_score,f1_score
import pickle
from imblearn.over_sampling import SMOTE


vocab_size = 30000
batch_size = 25
embedding_dim = 300
max_len = 3000

#loading the pickled test files
pickle_in = open("ht_test_text.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("ht_test_data.pickle","rb")
data = pickle.load(pickle_in)

test_X = pickle.load(open('/content/drive/My Drive/ML_Datasets/genData/test_text.pickle',"rb"))
test_data = pickle.load(open('/content/drive/My Drive/ML_Datasets/genData/test_data.pickle',"rb"))

#tokenizing the text
tokenizer = Tokenizer(num_words = vocab_size)
tokenizer.fit_on_texts(X)
word_index = tokenizer.word_index
train_sentences_tokenized = tokenizer.texts_to_sequences(X)

X = pad_sequences(train_sentences_tokenized, maxlen=max_len)

test_sentences_tokenized = tokenizer.texts_to_sequences(test_X)

test_X = pad_sequences(test_sentences_tokenized, maxlen=max_len)

tags = ['y','n']

label_enc = LabelBinarizer()
label_enc.fit(tags)
Y = label_enc.transform(data['cRUG'])

#oversampling using SMOTE
sm = SMOTE(random_state=4991, n_jobs=8, ratio={1:500, 0:500})
X,Y = sm.fit_sample(X,Y)
Y = to_categorical(Y)

test_Y = label_enc.transform(test_data['cRUG'])

test_Y = to_categorical(test_Y)

#loading the model
model5 = load_model('my_model_cRUG_tag');
a,b = 0,2
print('\n\nRuggedness')


#setting the layers non-trainable except the last 3 
for layer in model5.layers[:-3]:
    layer.trainable = False


model = Sequential()
model.add(model5)
model.summary()

#saving the model
model.save('DL_Final_RUG')
adam = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=0.0001, decay=0.0001)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy']) 
model.fit(X,Y,epochs = 100,batch_size = batch_size,validation_data = (test_X,test_Y))
pred = model.predict(test_X)
pred = pred.argmax(axis=1)
c_matrix = confusion_matrix(test_Y.argmax(axis=1),pred)
print(c_matrix)
accuracy = accuracy_score(test_Y.argmax(axis=1),pred)
print('Accuracy : ',accuracy)
# precision = true positive / total predicted positive(True positive + False positive)
# recall = true positive / total actual positive(True positive + False Negative)

#printing the classification metrics
print(classification_report(test_Y.argmax(axis=1),pred))
print(accuracy)
print(precision_score(test_Y.argmax(axis=1),pred))
print(recall_score(test_Y.argmax(axis=1),pred))
print(f1_score(test_Y.argmax(axis=1),pred))




/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/imblearn/utils/_validation.py:257: UserWarning: After over-sampling, the number of samples (500) in class 1 will be larger than the number of samples in the majority class (class #0 -> 310)
  n_samples_majority))
/usr/local/lib/python3.6/dist-packages/imblearn/utils/_validation.py:257: UserWarning: After over-sampling, the number of samples (500) in class

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


Ruggedness
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_2 (Model)              (None, 2)                 9148782   
Total params: 9,148,782
Trainable params: 8,386
Non-trainable params: 9,140,396
_________________________________________________________________
Train on 1000 samples, validate on 336 samples
Epoch 1/100
1000/1000 [==============================] - 5s 5ms/step - loss: 5.5198 - acc: 0.5410 - val_loss: 6.4877 - val_acc: 0.5357
Epoch 2/100
1000/1000 [==============================] - 1s 891us/step - loss: 5.5473 - acc: 0.5190 - val_loss: 6.4766 - val_acc: 0.5298
Epoch 3/100

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
